# OSC communication
With the OSC communication module of sc3nb you can directly send and receive OSC messages.

Open Sound Control (OSC) is a networking protocol for sound and is used by SuperCollider to communicate between sclang and scsynth. sc3nb is itself a OSC client and server. This allows sc3nb to send and receive OSC traffic.

For more information on OSC please refer to the following links:
* [Open Sound Control Specification](http://opensoundcontrol.org/spec-1_0)
* [Server vs Client SuperCollider Guide](http://doc.sccode.org/Guides/ClientVsServer.html)

In [ ]:
import sc3nb as scn

To see more information what messages are send and received set the logging level to INFO as seen below.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
# even more verbose logging is also avaible
# logging.basicConfig(level=logging.DEBUG)

In [ ]:
sc = scn.startup()

sc3nb serves as OSC server and as client of the SuperCollider server `scsynth`.<br>
You can also communicate with the SuperCollider interpreter `sclang`.<br>
You can see the current connection information with `sc.get_connection_info()`

In [ ]:
sc.get_connection_info()

## Types of messages
sc3nb is has 3 types of messages
* async messages
* message pairs
* all other messages


### async messages
* async messages are messages that are handled in an async fashion by scsynth.
* All async messages known to sc3nb are stored in `sc.async_msgs`

In [ ]:
sc.async_msgs

If msg is called with `sync=True` (default) sc3nb will perform a `sc.sync()` after the messages was send

In [ ]:
help(sc.sync)

Here is a example for the usage of `sc.msg` with an async message

In [ ]:
# /b_alloc has following arguments
# int    buffer number
# int    number of frames
# int    number of channels (optional. default = 1 channel)
# bytes  an OSC message to execute upon completion. (optional)
numFrames = 100
numChannels = 2
msg = scn.build_message("/s_new", ["s1", sc.nextNodeID(), 1, 0, "freq", 300])

sc.msg("/b_alloc", [sc.nextBufferID(), numFrames, numChannels, msg.dgram])

### Message Pairs
* Message pairs are messages that have a reply address. 
* All message pairs known by sc3nb are stored in `sc.msg_pairs`.
* These are the only messages that can be received. See below for more information on receiving messages and how to receive messages at custom OSC address 

In [ ]:
sc.msg_pairs

## Usage

### Sending

To send OSC messages you can use sc.msg

In [ ]:
help(sc.msg)

You can also send OSC bundles.

Currently sc3nb only supports bundles with exactly one message.
This is useful to provide timestamps for the server  

In [ ]:
help(sc.bundle)

### Receiving

When messages are received they are stored in one of the message queues of the OSC communication module of sc3nb

You can see all queues and thier OSC address with the help of `sc.msg_queues`. There is a queue for each message pair in `sc.msg_pairs`.

In [ ]:
sc.msg_queues

If we send a message to one of this addresses we receive the reply as return value

In [ ]:
sc.msg("/sync", 1500)

If we specify `sync=False` the message will be kept in the queue

In [ ]:
sc.msg("/sync", 42, sync=False)

In [ ]:
sc.msg_queues["/sync"]

You can see how many values were hold.

In [ ]:
sc.msg_queues["/sync"].skips

Notice that these hold messages can be skipped. 

In [ ]:
sc.sync()
sc.msg_queues["/sync"].skips

In [ ]:
sc.msg("/status", sync=False)

In [ ]:
sc.msg_queues["/status"]

In [ ]:
sc.msg("/status")

Therefore you should retrieve them with `get` if you want care for the queue values.

In [ ]:
sc.msg("/status", sync=False)

In [ ]:
sc.msg_queues["/status"].get()

In [ ]:
sc.msg("/status")

To add a new message queue you can simply use `sc.update_msg_queues()`

In [ ]:
help(sc.update_msg_queues)

In [ ]:
sc.update_msg_queues({"/test": "/test.reply", "/empty": "/empty.reply"})

This updates the `msg_pairs`

In [ ]:
sc.msg_pairs

You can now use the msg_queue of `/test`

In [ ]:
sc.msg_queues["/test"]

Lets use `OSCFunc` from SuperCollider to send us a reply when we send `/test` to `sclang`

In [ ]:
%%sc
OSCFunc.newMatching({|msg, time, addr, recvPort| addr.sendMsg("/test.reply", "Hello there!")}, '/test');

In [ ]:
sc.msg("/test", sclang=True)

In [ ]:
%%sc
OSCFunc.newMatching({|msg, time, addr, recvPort| addr.sendMsg("/empty.reply")}, '/empty');

In [ ]:
sc.msg("/empty", sclang=True, sync=False)
sc.msg("/empty", sclang=True, sync=False)

In [ ]:
sc.msg_queues["/empty"]

In [ ]:
print(sc.msg_queues["/empty"].get())

In [ ]:
sc.msg_queues["/empty"]